In [15]:
import pandas as pd
import codecs
import math

In [36]:
with codecs.open("data/countries.csv", "r", "utf-8", "ignore") as file:
    df = pd.read_table(file,encoding = "utf-8", delimiter=",")

# remove UM because it has no latitude or longitude
df = df[df['country']!='UM']
df

,country,latitude,longitude,name
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla
...,...,...,...,...
240,YE,15.552727,48.516388,Yemen
241,YT,-12.827500,45.166244,Mayotte
242,ZA,-30.559482,22.937506,South Africa
243,ZM,-13.133897,27.849332,Zambia


In [74]:
# calculate distance
def calc_distance(latitudeA, longitudeA, latitudeB, longitudeB, ):
    rlaA = math.radians(latitudeA)
    rloA = math.radians(longitudeA)
    
    rlaB = math.radians(latitudeB)
    rloB = math.radians(longitudeB)

    # 北極からの角度
    a = math.pi/2-rlaA
    b = math.pi/2-rlaB
    # 経度の差
    C = rloA - rloB

    return math.acos(max(min(math.cos(a) * math.cos(b) + math.sin(a) * math.sin(b) * math.cos(C), 1), -1))

In [75]:
country2lola = {}
for country, latitude, longitude in zip(list(df['country']), list(df['latitude']), list(df['longitude'])):
    country2lola[country] = (latitude, longitude)

In [79]:
distance_matrix = {}
for country_i in list(country2lola.keys()):
    distance_matrix_i = {}
    for country_j in list(country2lola.keys()):
        distance_matrix_i[country_j] = calc_distance(country2lola[country_i][0], country2lola[country_i][1], country2lola[country_j][0], country2lola[country_j][1])
    distance_matrix[country_i] = distance_matrix_i